In [60]:
print("Notebook is working!")

Notebook is working!


In [61]:
import os
print(os.listdir())

['.vscode', 'data', 'main.py', 'models', 'project.ipynb', 'src', 'venv']


In [62]:
import json
file_path="data/problems_data.jsonl"
data=[]
with open(file_path,"r",encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line.strip()))
len(data)

4112

In [63]:
data[1]

{'title': 'House Building',
 'description': 'A number of eccentrics from central New York have decided\n    that they have had enough of modern society, and want to move\n    from there. Together they have bought a rectangular piece of\n    land far away, and will now settle there.\nThe land consists of $N \\times\n    M$ squares, and it is possible to build a maximum of one\n    house on a given square. Each square has value $a_{x,y}$ that describes how nice it\n    is, on a scale between $0$\n    and $100$.\nThe goal of the eccentrics is to get as far away as possible\n    from everyone else, including each other. The happiness an\n    eccentric experiences from building his house on square\n    $(x,y)$ is thus\n    $a_{x,y}\\cdot d$, where\n    $d$ is the smallest\n    distance to another person.\nOut of habit, the eccentrics use Manhattan\n    distance to measure this; $d$ is defined as $\\min |x - x_2| + |y - y_2|$ over all\n    other people’s squares $(x_2,\n    y_2)$.\nThe eccen

In [64]:
data[0].keys()

dict_keys(['title', 'description', 'input_description', 'output_description', 'sample_io', 'problem_class', 'problem_score', 'url'])

In [65]:
import pandas as pd
df=pd.DataFrame(data)
df.head()

,title,description,input_description,output_description,sample_io,problem_class,problem_score,url
0,Uuu,Unununium (Uuu) was the name of the chemical\n...,The input consists of one line with two intege...,The output consists of $M$ lines where the $i$...,"[{'input': '7 10', 'output': '1 2 2 3 1 3 3 4 ...",hard,9.7,https://open.kattis.com/problems/uuu
1,House Building,A number of eccentrics from central New York h...,"The input consists of $10$ test cases, which a...",Print $K$ lines with\n the positions of the...,"[{'input': '0 2 3 2 50 60 50 30 50 40', 'outpu...",hard,9.7,https://open.kattis.com/problems/husbygge
2,Mario or Luigi,Mario and Luigi are playing a game where they ...,,,"[{'input': '', 'output': ''}]",hard,9.6,https://open.kattis.com/problems/marioorluigi
3,The Wire Ghost,Žofka is bending a copper wire. She starts wit...,The first line contains two integers $L$ and $...,The output consists of a single line consistin...,"[{'input': '4 3 3 C 2 C 1 C', 'output': 'GHOST...",hard,9.6,https://open.kattis.com/problems/thewireghost
4,Barking Up The Wrong Tree,"Your dog Spot is let loose in the park. Well, ...",The first line of input consists of two intege...,Write a single line containing the length need...,"[{'input': '2 0 10 0 10 10', 'output': '14.14'...",hard,9.6,https://open.kattis.com/problems/barktree


In [66]:
df["problem_class"].value_counts()

problem_class
hard      1941
medium    1405
easy       766
Name: count, dtype: int64

In [67]:
df.dtypes

title                  object
description            object
input_description      object
output_description     object
sample_io              object
problem_class          object
problem_score         float64
url                    object
dtype: object

In [68]:
df=df.fillna("")
df.isnull().sum()

title                 0
description           0
input_description     0
output_description    0
sample_io             0
problem_class         0
problem_score         0
url                   0
dtype: int64

In [69]:
df["text"]=(df["title"].astype(str)+" "+
            df["description"].astype(str)+" "+
            df["input_description"].astype(str)+" "+
            df["output_description"].astype(str)+" "+
            df["sample_io"].astype(str))
df["text"]=df["text"].str.lower()
df["text"]=df["text"].str.replace(r'\s+',' ',regex=True)
display(df["text"])

0       uuu unununium (uuu) was the name of the chemic...
1       house building a number of eccentrics from cen...
2       mario or luigi mario and luigi are playing a g...
3       the wire ghost žofka is bending a copper wire....
4       barking up the wrong tree your dog spot is let...
                              ...                        
4107    tölvunarfræðingar telja computer scientists co...
4108    velkomin! welcome to forritunarkeppni framhald...
4109    til hamingju there is no input in this problem...
4110    hipp hipp there is no input in this problem. p...
4111    advanced causal measurements causality is a ve...
Name: text, Length: 4112, dtype: object

In [70]:
df["problem_class"].value_counts()

problem_class
hard      1941
medium    1405
easy       766
Name: count, dtype: int64

In [71]:
X=df["text"]
y_class=df["problem_class"]
y_score=df["problem_score"]
from sklearn.model_selection import train_test_split
X_train, X_test, y_class_train, y_class_test=train_test_split(
    X, y_class, test_size=0.2, random_state=42
)
_, _, y_score_train, y_score_test=train_test_split(
    X, y_score, test_size=0.2, random_state=42
)

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer 
vectorizer=TfidfVectorizer(
    max_features=5000,
    stop_words="english"
)
X_train_tfidf=vectorizer.fit_transform(X_train)
X_test_tfidf=vectorizer.transform(X_test)
X_train_tfidf.shape

(3289, 5000)

In [73]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()
clf.fit(X_train_tfidf,y_class_train)
y_class_pred=clf.predict(X_test_tfidf)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
print("Accuracy: ",accuracy_score(y_class_test,y_class_pred))
print("\nClassification Report:\n",classification_report(y_class_test,y_class_pred))
print("\nConfusion Matrix:\n",confusion_matrix(y_class_test,y_class_pred))

Accuracy:  0.5054678007290401

Classification Report:
               precision    recall  f1-score   support

        easy       0.51      0.21      0.29       136
        hard       0.57      0.73      0.64       425
      medium       0.34      0.30      0.32       262

    accuracy                           0.51       823
   macro avg       0.48      0.41      0.42       823
weighted avg       0.49      0.51      0.48       823


Confusion Matrix:
 [[ 28  62  46]
 [ 12 310 103]
 [ 15 169  78]]


In [74]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
ridge=Ridge(alpha=1.0)
ridge.fit(X_train_tfidf, y_score_train)
y_score_pred=ridge.predict(X_test_tfidf)
mae=mean_absolute_error(y_score_test,y_score_pred)
rmse=np.sqrt(mean_squared_error(y_score_test,y_score_pred))
r2=r2_score(y_score_test,y_score_pred)
print("MAE:",mae)
print("RMSE:",rmse)
print("R2:",r2)



MAE: 1.7344701088010275
RMSE: 2.096985636245034
R2: 0.08393236933903492


In [75]:
import pickle
pickle.dump(vectorizer, open("models/tfidf_vectorizer.pkl", "wb"))
pickle.dump(clf, open("models/probclass_model.pkl", "wb"))
pickle.dump(ridge, open("models/probscore_model.pkl", "wb"))
def probl_predict(s):
    x=vectorizer.transform([s])
    diff=clf.predict(x)[0]
    score=ridge.predict(x)[0]
    return diff,score
sample = "Given an array find maximum subarray sum using Kadane algorithm"
probl_predict(sample)


('medium', np.float64(4.4115878664620425))